# Text Parser for Different File Formats 

In [ ]:
import pandas as pd
import os

## List files in Directory by file format

In [ ]:
# List of files in the specified directory by file format

# Update 'directory_in_str' and 'label' variables
directory_in_str = r"C:\Users\"
label = 0 

directory = os.fspath(directory_in_str)        

filesDocx = []
filesPdf = []
filesMsg = []
filesEml = []
filesXls = []
filesPpt = []
filesOther = []

for file in os.listdir(directory):
#     files.append(os.path.join(directory, file))    

    if file.endswith(".docx") or file.endswith(".doc") or file.endswith(".DOC"):
        filesDocx.append(os.path.join(directory, file))  
        continue
        
    if file.endswith(".pdf"):
        filesPdf.append(os.path.join(directory, file)) 
        continue
        
    if file.endswith(".msg"):
        filesMsg.append(os.path.join(directory, file)) 
        continue
    
    if file.endswith(".eml"): 
        filesEml.append(os.path.join(directory, file)) 
        continue
    
    if file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".xlsm") or file.endswith(".xlm"):
        filesXls.append(os.path.join(directory, file)) 
        continue
    
    if file.endswith(".pptx") or file.endswith(".ppt"):
        filesPpt.append(os.path.join(directory, file))  
        continue
        
    else: 
        filesOther.append(os.path.join(directory, file)) 
              
print("Docx files: %d" %len(filesDocx))
print("Pdf files: %d" %len(filesPdf))
print("Msg files: %d" %len(filesMsg))
print("Eml files: %d" %len(filesEml))
print("Xls files: %d" %len(filesXls))
print("Ppt files: %d" %len(filesPpt))
print("Other files: %d" %len(filesOther))

# files#[:10]

In [ ]:
filesOther#[:20]
# filesDocx[5]

## Parse Outlook .Msg messages to DataFrame

In [ ]:
import win32com.client
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [ ]:
df1 = pd.DataFrame(columns = ['Text', 'Label', 'FileType']) #'Subject', 'Date', 'Sender', 'Text'])

i=0
for email in filesMsg:
    
    msg = outlook.OpenSharedItem(email)
  
    df1.loc[i] = [msg.Body, 
                  label,
                 'email'
                 ] # msg.Subject msg.SenderName
    
    i = i+1
    del msg #reset msg instance to avoid error

outlook.Application.Quit()    

print(df1.shape)
df1.head()

# df.shape

## Parse Docouments to DataFrame

In [ ]:
word = win32com.client.Dispatch("Word.Application")

df2 = pd.DataFrame(columns = ['Text', 'Label', 'FileType']) 

i=0
for doc in filesDocx:
    
    wb = word.Documents.Open(doc)
    word.visible = False

    df2.loc[i] = [wb.Range().Text, 
                  label, 
                 'document'] 
    
    i = i+1
    del wb #reset msg instance to avoid error

word.Quit()

df2.head()

In [ ]:
# Check that the entire document is being parsed

print(df2.shape)
df2['Text'].apply(len)


## Parse PDFs to Dataframe

In [ ]:
import pdfminer 

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io

def pdfparser(data):

    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()

    return data

# pdfparser(filesPdf[3])

In [ ]:
df3 = pd.DataFrame(columns = ['Text', 'Label', 'FileType']) 

i=0
for doc in filesPdf:
    
    df3.loc[i] = [pdfparser(doc), 
                  label,
                 'pdf'] 
    i = i+1

df3

In [ ]:
# Check that the entire document is being parsed

print(df3.shape)
df3['Text'].apply(len)

## Concatenate Text into one DataFrame

In [ ]:
frames = [df1, df2, df3]

df4 = pd.concat(frames)

df4.shape

In [ ]:
# Save Df for use later - **CHANGE FILENAME**

filename = 'df.pkl'

df4.to_pickle(filename) 


## Alternative Directory Search

In [ ]:
import glob
from pathlib import Path

for file_path in Path(r'C:\Users\').glob('**/*.doc'):
    print(file_path)pyenv-virtualenv
    
# f = glob.glob('docs/*.doc') + glob.glob('docs/*.DOC')

outDir = r'C:\Users\'
if not os.path.exists(outDir):
    os.makedirs(outDir)
for i in f:
    os.system("catdoc -w '%s' > '%s'" %
              (i, outDir + '/' + re.sub(r'.*/([^.]+)\.doc', r'\1.txt', i, flags=re.IGNORECASE)))
